In [ ]:
# =============================================================================
# Middleware - Processing Messages Before and After LLM Calls (LangChain 1.0+)
# =============================================================================
"""
=====================================================================
Understanding Middleware in LangChain
=====================================================================

Middleware intercepts and processes messages before they reach the LLM
and after the LLM responds. Think of it like HTTP middleware in web apps.

Why Use Middleware?
-------------------
1. Context Management - Summarize long conversations to fit context windows
2. Logging - Track all LLM interactions
3. Caching - Store and retrieve common responses
4. Filtering - Block certain types of content
5. Transformation - Modify messages before/after LLM

Middleware Flow:
----------------
    User Input
         │
         ▼
    ┌─────────────────────┐
    │  Middleware Layer   │  ← Pre-process (summarize, filter, transform)
    │  (before LLM)       │
    └─────────────────────┘
         │
         ▼
    ┌─────────────────────┐
    │       LLM           │
    └─────────────────────┘
         │
         ▼
    ┌─────────────────────┐
    │  Middleware Layer   │  ← Post-process (log, cache, transform)
    │  (after LLM)        │
    └─────────────────────┘
         │
         ▼
    Final Response

Key Middleware Types (LangChain 1.0+):
--------------------------------------
- SummarizationMiddleware - Summarizes old messages when context gets long
- Custom middleware using RunnableLambda

Updated for LangChain 1.0+ (2025-2026)
"""

import os
from dotenv import load_dotenv

load_dotenv()

os.environ["OPENAI_API_KEY"] = os.getenv("OPENAI_API_KEY")
os.environ["GROQ_API_KEY"] = os.getenv("GROQ_API_KEY")
os.environ["GOOGLE_API_KEY"] = os.getenv("GOOGLE_API_KEY")

print("✅ Environment configured for middleware examples")

In [ ]:
# =============================================================================
# Custom Middleware with RunnableLambda (LangChain 1.0+)
# =============================================================================
"""
Building Custom Middleware
--------------------------
In LangChain 1.0+, you can create middleware-like behavior using:
1. RunnableLambda for transformations
2. Callbacks for logging/monitoring
3. Chain composition for pre/post processing

This approach gives you full control over the message flow.
"""

from langchain.chat_models import init_chat_model
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain_core.messages import SystemMessage, HumanMessage, AIMessage
from langchain_core.prompts import ChatPromptTemplate
from typing import List, Dict, Any
import time

# Initialize model
model = init_chat_model("gpt-4o-mini", model_provider="openai", temperature=0.7)

# =============================================================================
# Middleware 1: Logging Middleware
# =============================================================================
"""
Logs all inputs and outputs for debugging/monitoring.
"""

def logging_middleware(func_name: str):
    """Create a logging wrapper for any function."""
    def log_input(x):
        print(f"📥 [{func_name}] Input: {str(x)[:100]}...")
        return x
    
    def log_output(x):
        print(f"📤 [{func_name}] Output: {str(x)[:100]}...")
        return x
    
    return RunnableLambda(log_input), RunnableLambda(log_output)

# Create logging wrappers
log_in, log_out = logging_middleware("LLM_CALL")

# Chain with logging
logged_model = log_in | model | log_out

print("=" * 60)
print("Middleware 1: Logging Example")
print("=" * 60)

response = logged_model.invoke([HumanMessage(content="What is 2+2?")])

In [ ]:
# =============================================================================
# Middleware 2: Content Filtering Middleware
# =============================================================================
"""
Filters or transforms content before it reaches the LLM.
Useful for:
- Removing sensitive information
- Blocking certain topics
- Sanitizing user input
"""

# List of blocked words/topics (example)
BLOCKED_TOPICS = ["password", "credit card", "ssn", "social security"]

def content_filter(messages: List) -> List:
    """
    Pre-processing middleware that filters sensitive content.
    """
    filtered_messages = []
    
    for msg in messages:
        content = msg.content.lower()
        
        # Check for blocked topics
        blocked = [topic for topic in BLOCKED_TOPICS if topic in content]
        
        if blocked:
            # Replace with warning message
            print(f"⚠️  Blocked topics detected: {blocked}")
            filtered_messages.append(
                HumanMessage(content="[Content filtered - sensitive information removed]")
            )
        else:
            filtered_messages.append(msg)
    
    return filtered_messages

# Create filter middleware
filter_middleware = RunnableLambda(content_filter)

# Chain with filtering
filtered_model = filter_middleware | model

print("=" * 60)
print("Middleware 2: Content Filtering Example")
print("=" * 60)

# Test with safe content
print("\n✅ Safe content:")
safe_response = filtered_model.invoke([
    HumanMessage(content="What is the capital of France?")
])
print(f"   Response: {safe_response.content[:100]}...")

# Test with blocked content
print("\n⛔ Content with blocked topic:")
blocked_response = filtered_model.invoke([
    HumanMessage(content="What is my credit card number?")
])
print(f"   Response: {blocked_response.content[:100]}...")

In [ ]:
# =============================================================================
# Middleware 3: Message Trimming/Summarization
# =============================================================================
"""
Handles long conversations by trimming or summarizing old messages.
This is crucial for staying within context window limits.
"""

from langchain_core.messages import trim_messages

def context_manager(max_messages: int = 10):
    """
    Middleware that manages conversation context.
    Keeps system message + last N messages.
    """
    def trim_context(messages: List) -> List:
        if len(messages) <= max_messages:
            return messages
        
        # Separate system message from others
        system_msgs = [m for m in messages if isinstance(m, SystemMessage)]
        other_msgs = [m for m in messages if not isinstance(m, SystemMessage)]
        
        # Keep system + last (max_messages - 1) messages
        trimmed = system_msgs + other_msgs[-(max_messages - len(system_msgs)):]
        
        print(f"📋 Context trimmed: {len(messages)} → {len(trimmed)} messages")
        return trimmed
    
    return RunnableLambda(trim_context)

# Create context manager middleware
context_middleware = context_manager(max_messages=6)

print("=" * 60)
print("Middleware 3: Context Management Example")
print("=" * 60)

# Simulate a long conversation
long_conversation = [
    SystemMessage(content="You are a helpful assistant."),
]
for i in range(10):
    long_conversation.append(HumanMessage(content=f"Question {i+1}"))
    long_conversation.append(AIMessage(content=f"Answer {i+1}"))

print(f"\n📊 Original conversation: {len(long_conversation)} messages")

# Apply trimming middleware
trimmed = context_middleware.invoke(long_conversation)
print(f"📊 After middleware: {len(trimmed)} messages")
print(f"\n📋 Kept messages:")
for msg in trimmed:
    msg_type = type(msg).__name__[:6]
    print(f"   [{msg_type}] {msg.content[:40]}...")

In [ ]:
# =============================================================================
# Middleware 4: Response Transformation Middleware
# =============================================================================
"""
Post-processing middleware that transforms LLM responses.
Useful for:
- Formatting responses
- Adding metadata
- Extracting specific parts
"""

def response_formatter(format_type: str = "bullet"):
    """
    Post-processing middleware that formats responses.
    """
    def format_response(response: AIMessage) -> AIMessage:
        content = response.content
        
        if format_type == "uppercase":
            # Convert to uppercase (simple example)
            formatted = content.upper()
        elif format_type == "with_timestamp":
            # Add timestamp
            import datetime
            timestamp = datetime.datetime.now().strftime("%Y-%m-%d %H:%M:%S")
            formatted = f"[{timestamp}]\n{content}"
        elif format_type == "word_count":
            # Add word count
            word_count = len(content.split())
            formatted = f"{content}\n\n---\n📊 Word count: {word_count}"
        else:
            formatted = content
        
        return AIMessage(content=formatted)
    
    return RunnableLambda(format_response)

print("=" * 60)
print("Middleware 4: Response Transformation Example")
print("=" * 60)

# Create different formatters
timestamp_formatter = response_formatter("with_timestamp")
wordcount_formatter = response_formatter("word_count")

# Chain with formatting
formatted_model = model | wordcount_formatter

response = formatted_model.invoke([
    HumanMessage(content="Explain what Python is in 2 sentences.")
])

print(f"\n📝 Formatted Response:")
print(response.content)

In [ ]:
# =============================================================================
# Middleware 5: Chaining Multiple Middleware
# =============================================================================
"""
Combining Multiple Middleware Layers
------------------------------------
You can chain multiple middleware together for complex processing.
Order matters: they execute left to right for input, right to left for output.
"""

def timing_middleware():
    """Middleware that measures LLM call duration."""
    start_time = None
    
    def start_timer(x):
        nonlocal start_time
        start_time = time.time()
        print("⏱️  Starting LLM call...")
        return x
    
    def end_timer(x):
        duration = time.time() - start_time
        print(f"⏱️  LLM call completed in {duration:.2f}s")
        return x
    
    return RunnableLambda(start_timer), RunnableLambda(end_timer)

def system_prompt_injector(system_content: str):
    """Middleware that ensures a system message is always present."""
    def inject(messages: List) -> List:
        has_system = any(isinstance(m, SystemMessage) for m in messages)
        if not has_system:
            print("💉 Injecting system message")
            return [SystemMessage(content=system_content)] + messages
        return messages
    return RunnableLambda(inject)

print("=" * 60)
print("Middleware 5: Chained Middleware Example")
print("=" * 60)

# Create middleware components
timer_start, timer_end = timing_middleware()
system_injector = system_prompt_injector("You are a concise assistant. Keep answers brief.")

# Build the full pipeline:
# inject system → filter → start timer → model → end timer → format
full_pipeline = (
    system_injector 
    | filter_middleware 
    | timer_start 
    | model 
    | timer_end 
    | wordcount_formatter
)

print("\n🔗 Full middleware chain:")
print("   system_injector → filter → timer_start → model → timer_end → formatter")

# Test the full pipeline
response = full_pipeline.invoke([
    HumanMessage(content="What is machine learning?")
])

print(f"\n📝 Final Response:\n{response.content}")

In [ ]:
# =============================================================================
# SummarizationMiddleware with create_agent (LangChain Agents)
# =============================================================================
"""
Built-in SummarizationMiddleware
--------------------------------
LangChain provides SummarizationMiddleware for agents that automatically
summarizes old conversation turns when context gets long.

This is used with create_agent() and requires:
- A checkpointer for state persistence
- Configuration for when to trigger summarization

Note: This requires langgraph for checkpointing functionality.
"""

from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware

# Note: This example shows the pattern. You need langgraph installed for checkpointing.
print("=" * 60)
print("SummarizationMiddleware Pattern (for Agents)")
print("=" * 60)

print("""
The SummarizationMiddleware is used with create_agent():

```python
from langchain.agents import create_agent
from langchain.agents.middleware import SummarizationMiddleware
from langgraph.checkpoint.memory import InMemorySaver

agent = create_agent(
    model="gpt-4o-mini",
    tools=[your_tools],
    checkpointer=InMemorySaver(),  # Required for state
    middleware=[
        SummarizationMiddleware(
            model="gpt-4o-mini",          # Model for summarization
            trigger=("tokens", 4000),     # Trigger at 4000 tokens
            keep=("messages", 20),        # Keep last 20 messages
        ),
    ],
)

# Use with thread_id for conversation persistence
config = {"configurable": {"thread_id": "user_123"}}
response = agent.invoke({"messages": [HumanMessage(...)]}, config=config)
```

How it works:
1. Monitors conversation token count
2. When trigger threshold is reached:
   - Summarizes older messages
   - Keeps recent messages intact
   - Replaces old messages with summary
3. Keeps conversation within context limits
""")

# =============================================================================
# Practical Example: Conversation with Auto-Summarization
# =============================================================================
"""
Without the full agent setup, here's how to implement similar
functionality using pure middleware:
"""

def summarization_middleware(model, token_threshold: int = 1000):
    """
    Middleware that summarizes old messages when context gets long.
    """
    def maybe_summarize(messages: List) -> List:
        # Simple token estimation (actual implementation would use tiktoken)
        estimated_tokens = sum(len(m.content.split()) * 1.3 for m in messages)
        
        if estimated_tokens < token_threshold:
            return messages
        
        print(f"📝 Context too long (~{int(estimated_tokens)} tokens). Summarizing...")
        
        # Separate system from conversation
        system_msgs = [m for m in messages if isinstance(m, SystemMessage)]
        conv_msgs = [m for m in messages if not isinstance(m, SystemMessage)]
        
        # Keep last 4 messages, summarize the rest
        to_summarize = conv_msgs[:-4]
        to_keep = conv_msgs[-4:]
        
        if to_summarize:
            # Create summary of old messages
            summary_prompt = f"Summarize this conversation briefly:\n\n"
            for msg in to_summarize:
                role = "User" if isinstance(msg, HumanMessage) else "Assistant"
                summary_prompt += f"{role}: {msg.content}\n"
            
            summary = model.invoke([HumanMessage(content=summary_prompt)])
            
            # Create new message list
            summary_msg = SystemMessage(
                content=f"[Previous conversation summary: {summary.content}]"
            )
            
            return system_msgs + [summary_msg] + to_keep
        
        return messages
    
    return RunnableLambda(maybe_summarize)

print("\n📋 Custom summarization middleware created")

In [ ]:
# =============================================================================
# Summary: Middleware in LangChain 1.0+
# =============================================================================
"""
=====================================================================
KEY TAKEAWAYS - Middleware in LangChain
=====================================================================

1. MIDDLEWARE CONCEPT:
   -------------------
   Middleware intercepts messages before/after LLM calls.
   Use RunnableLambda to create custom middleware.

2. COMMON MIDDLEWARE PATTERNS:
   ---------------------------
   # Logging
   def log(x): print(x); return x
   logged_chain = RunnableLambda(log) | model
   
   # Filtering
   def filter(msgs): return [m for m in msgs if is_safe(m)]
   filtered_chain = RunnableLambda(filter) | model
   
   # Transformation
   def transform(response): return modify(response)
   transformed_chain = model | RunnableLambda(transform)

3. CHAINING MIDDLEWARE:
   --------------------
   pipeline = (
       preprocessing_middleware
       | model
       | postprocessing_middleware
   )

4. BUILT-IN SUMMARIZATION:
   ------------------------
   from langchain.agents.middleware import SummarizationMiddleware
   
   agent = create_agent(
       model="gpt-4o-mini",
       middleware=[
           SummarizationMiddleware(
               model="gpt-4o-mini",
               trigger=("tokens", 4000),
               keep=("messages", 20),
           ),
       ],
   )

5. BEST PRACTICES:
   ----------------
   - Keep middleware focused (single responsibility)
   - Log inputs/outputs for debugging
   - Handle errors gracefully
   - Consider performance impact
   - Test middleware in isolation

Common Imports:
---------------
from langchain_core.runnables import RunnableLambda, RunnablePassthrough
from langchain.agents.middleware import SummarizationMiddleware

=====================================================================
"""

print("=" * 60)
print("Middleware Module Complete!")
print("=" * 60)
print("""
Next Topics to Explore:
-----------------------
1. 7-LCEL.ipynb - LangChain Expression Language deep dive
2. 8-streaming.ipynb - Streaming and async patterns
3. 9-RAG.ipynb - Retrieval Augmented Generation
4. 10-agents.ipynb - Agent patterns with create_agent()

Middleware Use Cases:
---------------------
- Rate limiting and throttling
- Caching frequent responses
- Input sanitization
- Response formatting
- Analytics and monitoring
- A/B testing different prompts
""")